Reference
- https://wikidocs.net/24996

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/spa-eng.zip'
filename = 'spa-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
lines= pd.read_csv('spa.txt', names=['src', 'tar', 'what'], sep='\t') 
len(lines)

125446

In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:100000] # 10만개만 저장
lines.sample(10)

,src,tar
63698,Can't you see we're very busy?,¿No ves que estamos muy ocupados?
40050,These scissors cut well.,Estas tijeras cortan bien.
77653,It is worth visiting that museum.,Vale la pena visitar ese museo.
5916,Are you coming?,¿Vienes?
17402,I couldn't bear it.,No pude soportarlo.
93906,"On July tenth, the veto was announced.","El diez de julio, el veto fue anunciado."
43448,I'm sorry I bothered you.,Siento haberte importunado.
11982,My house is tiny.,Mi casa es minúscula.
40293,Tom died in an accident.,Tom murió en un accidente.
31829,We continued chatting.,Seguimos hablando.


In [54]:
# 문장의 시작과 끝을 나타내는 토큰 설정
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
51819,I should have left earlier.,\t Debería haberme ido antes. \n
89821,Do you know if Tom has already eaten?,\t ¿Sabe usted si Tom ya ha comido? \n
17664,I like watching TV.,\t Me gusta ver televisión. \n
32795,Do you speak Esperanto?,\t ¿Hablas esperanto? \n
78882,Tom doesn't like classical music.,\t A Tom no le gusta la música clásica. \n
44060,She doesn't need to work.,\t Ella no tiene que trabajar. \n
36697,Where does that bus go?,\t ¿Hacia dónde va el colectivo? \n
23924,Where's your mother?,\t ¿Dónde está tu madre? \n
63087,We've been talking about you.,\t Hemos estado hablando sobre ti. \n
65333,I've lived here my whole life.,\t He vivido aquí mi vida entera. \n


In [56]:
# 영어 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩
    for char in line: # 1글자씩
        src_vocab.add(char) #set에는 add를 통해 자료 추가
#training set에서 나타난 알파벳들을 저장

# 스페인어 글자 집합 구축
tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [57]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print("영어 글자 집합:", src_vocab_size)
print("스페인어 글자 집합:", tar_vocab_size)

영어 글자 집합: 87
스페인어 글자 집합: 105


In [58]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u']


In [59]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)
#각 알파벳들에 index 부여

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '\xa0': 76, '°': 77, 'á': 78, 'ã': 79, 'è': 80, 'é': 81, 'ö': 82, '‘': 83, '’': 84, '₂': 85, '€': 86}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, "'": 8, '(': 9, ')': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';'

# 정수 인코딩
- 입력 데이터와 출력 데이터 모두를 정수 인코딩

In [60]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])
print(lines.src[:5])

[[30, 64, 9], [30, 64, 9], [30, 64, 9], [30, 64, 9], [31, 58, 9]]
0    Go.
1    Go.
2    Go.
3    Go.
4    Hi.
Name: src, dtype: object


In [61]:
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])
print(lines.tar[:5])

[[1, 3, 50, 59, 14, 3, 2], [1, 3, 50, 59, 74, 59, 14, 3, 2], [1, 3, 50, 55, 79, 55, 14, 3, 2], [1, 3, 50, 91, 79, 55, 73, 59, 14, 3, 2], [1, 3, 36, 69, 66, 55, 14, 3, 2]]
0        \t Ve. \n
1      \t Vete. \n
2      \t Vaya. \n
3    \t Váyase. \n
4      \t Hola. \n
Name: tar, dtype: object


In [82]:
#번역 결과에 시작과 끝을 나타내는 토큰은 필요하지 않다.
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])
print(lines.tar[:5].apply(lambda x: re.sub('\t','',x)).apply(lambda x: re.sub('\n','', x)))

[[3, 50, 59, 14, 3, 2], [3, 50, 59, 74, 59, 14, 3, 2], [3, 50, 55, 79, 55, 14, 3, 2], [3, 50, 91, 79, 55, 73, 59, 14, 3, 2], [3, 36, 69, 66, 55, 14, 3, 2]]
0         Ve. 
1       Vete. 
2       Vaya. 
3     Váyase. 
4       Hola. 
Name: tar, dtype: object


In [83]:
#패딩값을 주기 위해 최대 길이를 조절
#영어의 최대 길이와 스페인어의 최대 길이를 맞춰줄 필요는 없다. 영어는 영어끼리, 스페인어는 스페인어끼리
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('영어의 최대길이:', max_src_len)
print('스페인어의 최대길이:', max_tar_len)

영어의 최대길이: 41
스페인어의 최대길이: 96


In [84]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [85]:
#모든 값에 대하여 one_hot 인코딩을 수행
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [86]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
#인코더 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True/ 인코더에 입력을 넣으면 내부 상태를 리턴
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태(h)와 셀(c) 상태. 은닉상태와 셀 상태 두가지를 전달
#encoder_state에 은닉상태와 셀 상태 두가지를 저장하고, 이를 디코더에 전달하여 두 가지 상태 모두를 디코더로 전달

In [87]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 마지막 은닉 상태, 셀 상태로. 이에 따라 initial_state를 encoder_state로
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

In [89]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
#val_accuracy가 감소하게 되면 자동으로 epoch를 멈추는 earlystopping 사용

In [90]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2, callbacks=[callback])

Epoch 1/50
1250/1250 [==============================] - 25s 20ms/step - loss: 0.5985 - accuracy: 0.8276 - val_loss: 0.6783 - val_accuracy: 0.7943
Epoch 2/50
1250/1250 [==============================] - 25s 20ms/step - loss: 0.3926 - accuracy: 0.8810 - val_loss: 0.5628 - val_accuracy: 0.8307
Epoch 3/50
1250/1250 [==============================] - 24s 20ms/step - loss: 0.3345 - accuracy: 0.8992 - val_loss: 0.5112 - val_accuracy: 0.8464
Epoch 4/50
1250/1250 [==============================] - 24s 20ms/step - loss: 0.3040 - accuracy: 0.9084 - val_loss: 0.4795 - val_accuracy: 0.8559
Epoch 5/50
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2844 - accuracy: 0.9140 - val_loss: 0.4604 - val_accuracy: 0.8614
Epoch 6/50
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2705 - accuracy: 0.9181 - val_loss: 0.4486 - val_accuracy: 0.8644
Epoch 7/50
1250/1250 [==============================] - 25s 20ms/step - loss: 0.2598 - accuracy: 0.9213 - val_loss: 0.4396 -

In [91]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [92]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [93]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [94]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [101]:
for seq_index in range(100): # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장:  Ve. 
번역기가 번역한 문장:  Vete. 
-----------------------------------
입력 문장: Go.
정답 문장:  Vete. 
번역기가 번역한 문장:  Vete. 
-----------------------------------
입력 문장: Go.
정답 문장:  Vaya. 
번역기가 번역한 문장:  Vete. 
-----------------------------------
입력 문장: Go.
정답 문장:  Váyase. 
번역기가 번역한 문장:  Vete. 
-----------------------------------
입력 문장: Hi.
정답 문장:  Hola. 
번역기가 번역한 문장:  Disculpa. 
-----------------------------------
입력 문장: Run!
정답 문장:  ¡Corre! 
번역기가 번역한 문장:  ¡Colvido! 
-----------------------------------
입력 문장: Run!
정답 문장:  ¡Corran! 
번역기가 번역한 문장:  ¡Colvido! 
-----------------------------------
입력 문장: Run!
정답 문장:  ¡Corra! 
번역기가 번역한 문장:  ¡Colvido! 
-----------------------------------
입력 문장: Run!
정답 문장:  ¡Corred! 
번역기가 번역한 문장:  ¡Colvido! 
-----------------------------------
입력 문장: Run.
정답 문장:  Corred. 
번역기가 번역한 문장:  Conseguelo. 
-----------------------------------
입력 문장: Who?
정답 문장:  ¿Quién? 
번역기가 번역한 문장:  ¿A quién confiese? 
----------------------------

-----------------------------------
입력 문장: Listen.
정답 문장:  Escucha. 
번역기가 번역한 문장:  Escuchen. 
-----------------------------------
입력 문장: Listen.
정답 문장:  Escuche. 
번역기가 번역한 문장:  Escuchen. 
-----------------------------------
입력 문장: Listen.
정답 문장:  Escuchen. 
번역기가 번역한 문장:  Escuchen. 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡No puede ser! 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  De ninguna manera. 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡De ninguna manera! 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡Imposible! 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡De ningún modo! 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡De eso nada! 
번역기가 번역한 문장:  ¡Por favor! 
-----------------------------------
입력 문장: No way!
정답 문장:  ¡Ni cagando! 
번역기가 번역한 문장:  ¡Por favor!

- 번역된 결과는 틀린 답을 내긴 했으나, 문장의 구성요소를 갖추고 있고, 그 자체로 의미를 지닌 문장임
---
- 입력 문장: Stop! / 정답 문장:  ¡Pare! / 번역기가 번역한 문장:  ¡Pare!
- 입력 문장: Stop! / 정답 문장:  ¡Para! / 번역기가 번역한 문장:  ¡Pare! 
- 입력 문장: Wait. / 정답 문장:  Esperen. / 번역기가 번역한 문장:  Espera.  
-- 하나의 영어 단어, 문장에는 하나의 스페인어 답이 배정된다.  
-- 스페인어는 인칭 변화에 따라 동사의 형태가 변화하는 반면, 영어의 경우는 변화가 없기에 발생하는 문제

# 단어 단위

In [106]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile

In [107]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'spa-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [108]:
num_samples = 100000
#샘플 사용량

In [109]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [103]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [110]:
'''
교사 강요 (Teacher Forcing)을 위해 훈련에 사용할 디코더의 입력 시퀀스와
실제 값에 해당하는 출력 시퀀스를 따로 분리하여 저장.
입력 시퀀스에는 <SOS>를, 출력 시퀀스에는 <EOS>
'''

def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("spa.txt", "r", encoding='UTF8') as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [111]:
sents_en_in, sents_spa_in, sents_spa_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_spa_in[:5])
print(sents_spa_out[:5])

[['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
[['<sos>', 've', '.'], ['<sos>', 'vete', '.'], ['<sos>', 'vaya', '.'], ['<sos>', 'vayase', '.'], ['<sos>', 'hola', '.']]
[['ve', '.', '<eos>'], ['vete', '.', '<eos>'], ['vaya', '.', '<eos>'], ['vayase', '.', '<eos>'], ['hola', '.', '<eos>']]


In [112]:
sents_spa_in[:100]

[['<sos>', 've', '.'],
 ['<sos>', 'vete', '.'],
 ['<sos>', 'vaya', '.'],
 ['<sos>', 'vayase', '.'],
 ['<sos>', 'hola', '.'],
 ['<sos>', 'corre', '!'],
 ['<sos>', 'corran', '!'],
 ['<sos>', 'corra', '!'],
 ['<sos>', 'corred', '!'],
 ['<sos>', 'corred', '.'],
 ['<sos>', 'quien', '?'],
 ['<sos>', 'orale', '!'],
 ['<sos>', 'fuego', '!'],
 ['<sos>', 'incendio', '!'],
 ['<sos>', 'disparad', '!'],
 ['<sos>', 'ayuda', '!'],
 ['<sos>', 'socorro', '!', 'auxilio', '!'],
 ['<sos>', 'auxilio', '!'],
 ['<sos>', 'salta', '!'],
 ['<sos>', 'salte', '.'],
 ['<sos>', 'parad', '!'],
 ['<sos>', 'para', '!'],
 ['<sos>', 'pare', '!'],
 ['<sos>', 'espera', '!'],
 ['<sos>', 'esperen', '.'],
 ['<sos>', 'continua', '.'],
 ['<sos>', 'continue', '.'],
 ['<sos>', 'hola', '.'],
 ['<sos>', 'date', 'prisa', '!'],
 ['<sos>', 'daos', 'prisa', '!'],
 ['<sos>', 'dese', 'prisa', '.'],
 ['<sos>', 'me', 'oculte', '.'],
 ['<sos>', 'me', 'escondi', '.'],
 ['<sos>', 'me', 'ocultaba', '.'],
 ['<sos>', 'me', 'escondia', '.'],
 ['

In [114]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_spa = Tokenizer(filters="", lower=False)
tokenizer_spa.fit_on_texts(sents_spa_in)
tokenizer_spa.fit_on_texts(sents_spa_out)

decoder_input = tokenizer_spa.texts_to_sequences(sents_spa_in)
decoder_target = tokenizer_spa.texts_to_sequences(sents_spa_out)

In [115]:
decoder_input

[[2, 302, 1],
 [2, 1052, 1],
 [2, 479, 1],
 [2, 3932, 1],
 [2, 1504, 1],
 [2, 1317, 61],
 [2, 6014, 61],
 [2, 7128, 61],
 [2, 6015, 61],
 [2, 6015, 1],
 [2, 60, 8],
 [2, 3431, 61],
 [2, 533, 61],
 [2, 1446, 61],
 [2, 7129, 61],
 [2, 175, 61],
 [2, 7130, 61, 7131, 61],
 [2, 7131, 61],
 [2, 6016, 61],
 [2, 6017, 1],
 [2, 8788, 61],
 [2, 32, 61],
 [2, 2162, 61],
 [2, 483, 61],
 [2, 3039, 1],
 [2, 2163, 1],
 [2, 3225, 1],
 [2, 1504, 1],
 [2, 2164, 794, 61],
 [2, 8789, 794, 61],
 [2, 7132, 794, 1],
 [2, 15, 11933, 1],
 [2, 15, 5276, 1],
 [2, 15, 11934, 1],
 [2, 15, 7133, 1],
 [2, 2456, 1],
 [2, 3933, 1],
 [2, 19, 606, 1],
 [2, 64, 1582, 61],
 [2, 2457, 6, 61],
 [2, 8790, 25, 6018, 1],
 [2, 533, 61],
 [2, 7129, 61],
 [2, 11935, 61],
 [2, 8791, 61],
 [2, 8791, 61],
 [2, 3934, 61],
 [2, 3226, 1],
 [2, 36, 2086, 61],
 [2, 11936, 61],
 [2, 2086, 61],
 [2, 11937, 61],
 [2, 11938, 61],
 [2, 1222, 1],
 [2, 302, 78, 155, 1],
 [2, 3935, 78, 155, 1],
 [2, 479, 78, 155, 1],
 [2, 2349, 78, 155, 1],
 [2,

In [116]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [117]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_spa.word_index) + 1
print("영어 단어 집합의 크기 : {:d}".format(src_vocab_size))
print("스페인어 단어 집합의 크기 : {:d}".format(tar_vocab_size))

영어 단어 집합의 크기 : 10459
스페인어 단어 집합의 크기 : 20329


In [119]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_spa.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_spa.index_word # 훈련 후 결과 비교할 때 사용

In [120]:
indices = np.arange(encoder_input.shape[0], dtype=int)
np.random.shuffle(indices)
print(indices)

[12432 60449 91592 ... 92099 18998 89774]


In [122]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [123]:
#훈련 데이터와 테스트 데이터 분리
n_of_val = int(100000*0.1)
print(n_of_val) 

10000


In [127]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [128]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(90000, 13)
(90000, 17)
(90000, 17)
(10000, 13)
(10000, 17)
(10000, 17)


In [131]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [129]:
latent_dim = 50

In [132]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [133]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [134]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [135]:
#원핫 인코딩을 하지 않은 상태로 정수 레이블에 다중 클래스 분류 = sparse_categorical_crossentropy 사용
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [136]:
model.summary()

Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     522950      input_12[0][0]                   
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     1016450     input_13[0][0]                   
______________________________________________________________________________________

In [139]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)

In [140]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 64, epochs = 50, callbacks=[callback]) #early stopping 사용

Epoch 1/50
1407/1407 [==============================] - 95s 67ms/step - loss: 1.6207 - acc: 0.7564 - val_loss: 1.5852 - val_acc: 0.7621
Epoch 2/50
1407/1407 [==============================] - 95s 68ms/step - loss: 1.5116 - acc: 0.7702 - val_loss: 1.4959 - val_acc: 0.7746
Epoch 3/50
1407/1407 [==============================] - 96s 68ms/step - loss: 1.4319 - acc: 0.7818 - val_loss: 1.4362 - val_acc: 0.7830
Epoch 4/50
1407/1407 [==============================] - 93s 66ms/step - loss: 1.3743 - acc: 0.7912 - val_loss: 1.3900 - val_acc: 0.7902
Epoch 5/50
1407/1407 [==============================] - 92s 65ms/step - loss: 1.3259 - acc: 0.7993 - val_loss: 1.3596 - val_acc: 0.7964
Epoch 6/50
1407/1407 [==============================] - 92s 65ms/step - loss: 1.2937 - acc: 0.8060 - val_loss: 1.3397 - val_acc: 0.8028
Epoch 7/50
1407/1407 [==============================] - 94s 67ms/step - loss: 1.2820 - acc: 0.8122 - val_loss: 1.3408 - val_acc: 0.8071
Epoch 8/50
1407/1407 [==========================

In [141]:
encoder_model = Model(encoder_inputs, encoder_states)

In [142]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [143]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [144]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [145]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [147]:
for seq_index in range(100):
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  the room is dark . 
번역문 : la habitacion es oscura . 
예측문 :  la habitacion esta con la habitacion . 


원문 :  i go into the city every day . 
번역문 : yo voy a la ciudad todos los dias . 
예측문 :  me voy a la ciudad al dia . 


원문 :  there is no telling what will happen . 
번역문 : no se sabe que va a pasar . 
예측문 :  no hay todo lo que quiere . 


원문 :  he went to the shop . 
번역문 : ha ido a la tienda . 
예측문 :  fue a la tienda . 


원문 :  tom and i chatted for a while . 
번역문 : tom y yo charlamos durante un rato . 
예측문 :  tom y yo yo yo tambien estaba en la puerta . 


원문 :  would you like another apple ? 
번역문 : quieres otra manzana ? 
예측문 :  te gustaria otra vez ? 


원문 :  he eats lunch at a cafeteria . 
번역문 : el almuerza en una cafeteria . 
예측문 :  el como a un buen la manana . 


원문 :  don t change the channel . 
번역문 : no cambies de cadena . 
예측문 :  no en el ellos ? 


원문 :  tom traveled back in time . 
번역문 : tomas viajo al pasado . 
예측문 :  tom ha ido a tiempo . 


원문 :  tom is around thirt

원문 :  you should not wear a fur coat . 
번역문 : no deberias llevar un abrigo de piel . 
예측문 :  no deberias un ojos para mi solo . 


원문 :  i arrived later than usual . 
번역문 : yo llegue mas tarde de lo acostumbrado . 
예측문 :  me siento mas tarde . 


원문 :  listen to this . 
번역문 : escuche esto . 
예측문 :  esto en una decision . 


원문 :  she seems to get fatter and fatter . 
번역문 : parece estar poniendose cada vez mas gorda . 
예측문 :  parece que tiene que estar en casa demasiado r


원문 :  i wrote you three letters . 
번역문 : te escribi tres cartas . 
예측문 :  te vi a un poco para el tiempo . 


원문 :  how many hours are left ? 
번역문 : cuantas horas faltan ? 
예측문 :  cuantos horas ? 


원문 :  he usually comes in time . 
번역문 : el normalmente viene puntual . 
예측문 :  el tiempo para la hora . 


원문 :  all right . i ll accept your offer . 
번역문 : de acuerdo . acepto tu oferta . 
예측문 :  todo el mundo es idea de que se va a dar un buen d


원문 :  tom shouldn t have made mary angry . 
번역문 : tom no debio haber hech